In [1]:
import os
import sys
import gensim
import pandas as pd
import re
import pickle
from nltk.tokenize import RegexpTokenizer
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from sklearn.cross_validation import train_test_split
from gensim.models.word2vec import Word2Vec
from tqdm import tqdm
from sklearn import utils
import numpy as np
from keras import optimizers
from keras.models import load_model
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score, classification_report
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, GRU, Bidirectional

C:\Users\User\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\User\Anaconda3\lib\site-packages\pattern-2.6-py3.6.egg\pattern\text\en\..\..\..\..\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using 

In [2]:
def preprocess(text):
    text = re.sub(r'[^\w\s]','',text)
    text = re.sub(r'\d+','<number>',text)
    return text

In [3]:
clean_train_comments = pd.read_csv("./corpus/prosa/data_clean_punctuation/train.csv")
# clean_train_comments = pd.read_csv("./corpus/prosa/data_clean_punctuation/data_train_full.csv")
clean_train_comments['content'] = clean_train_comments['content'].apply(preprocess)
clean_train_comments.head()

,content,polarity
0,gerindra alihkan rekomendasi ke agus an tanri ...,neutral
1,cuci tangan pakai sunlight stelah itu pakai sa...,neutral
2,kasus toko obat digerebek fpi propam akan peri...,neutral
3,menkeu melemah nya rupiah lebih berpengaruh pa...,neutral
4,minyak jarak castor oil <number> ml,neutral


In [4]:
tokenizer = RegexpTokenizer(r'\w+')
clean_train_comments['content'] = clean_train_comments['content'].astype('str') 
clean_train_comments.dtypes
clean_train_comments['tokens'] = clean_train_comments['content'].apply(tokenizer.tokenize)
clean_train_comments['sentiment'] = clean_train_comments['polarity'].astype('category').cat.codes
   
clean_train_comments.head()

,content,polarity,tokens,sentiment
0,gerindra alihkan rekomendasi ke agus an tanri ...,neutral,"[gerindra, alihkan, rekomendasi, ke, agus, an,...",1
1,cuci tangan pakai sunlight stelah itu pakai sa...,neutral,"[cuci, tangan, pakai, sunlight, stelah, itu, p...",1
2,kasus toko obat digerebek fpi propam akan peri...,neutral,"[kasus, toko, obat, digerebek, fpi, propam, ak...",1
3,menkeu melemah nya rupiah lebih berpengaruh pa...,neutral,"[menkeu, melemah, nya, rupiah, lebih, berpenga...",1
4,minyak jarak castor oil <number> ml,neutral,"[minyak, jarak, castor, oil, number, ml]",1


In [5]:
clean_test_comments = pd.read_csv("./corpus/prosa/data_clean_punctuation/test.csv")
# clean_test_comments = pd.read_csv("./corpus/prosa/data_clean_punctuation/data_testing_full.csv")
clean_test_comments['content'] = clean_test_comments['content'].apply(preprocess)
clean_test_comments.head()

,content,polarity
0,kemarin gue datang ke tempat makan baru yang a...,negative
1,kayak nya sih gue tidak akan mau balik lagi ke...,negative
2,kalau dipikirpikir sebenarnya tidak ada yang b...,negative
3,ini pertama kalinya gua ke bank buat ngurusin ...,negative
4,waktu sampai dengan gue pernah disuruh ibu lat...,negative


In [6]:
clean_test_comments['content'] = clean_test_comments['content'].astype('str') 
clean_test_comments.dtypes
clean_test_comments["tokens"] = clean_test_comments["content"].apply(tokenizer.tokenize)
clean_test_comments['sentiment'] = clean_test_comments['polarity'].astype('category').cat.codes

clean_test_comments.head()

,content,polarity,tokens,sentiment
0,kemarin gue datang ke tempat makan baru yang a...,negative,"[kemarin, gue, datang, ke, tempat, makan, baru...",0
1,kayak nya sih gue tidak akan mau balik lagi ke...,negative,"[kayak, nya, sih, gue, tidak, akan, mau, balik...",0
2,kalau dipikirpikir sebenarnya tidak ada yang b...,negative,"[kalau, dipikirpikir, sebenarnya, tidak, ada, ...",0
3,ini pertama kalinya gua ke bank buat ngurusin ...,negative,"[ini, pertama, kalinya, gua, ke, bank, buat, n...",0
4,waktu sampai dengan gue pernah disuruh ibu lat...,negative,"[waktu, sampai, dengan, gue, pernah, disuruh, ...",0


In [7]:
x_train = clean_train_comments['content']
x_validation = clean_test_comments['content']
y_train = clean_train_comments['sentiment']
y_validation = clean_test_comments['sentiment']

In [8]:
def labelize_text(text,label):
    result = []
    prefix = label
    for i, t in zip(text.index, text):
        result.append(LabeledSentence(t.split(), [prefix + '_%s' % i]))
    return result
  
x_train = labelize_text(x_train, 'TRAIN')
x_validation = labelize_text(x_validation, 'TEST')

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  """


In [9]:
MAX_SEQUENCE_LENGTH = 85
data_dim = 700

In [10]:
# word2vec = Word2Vec.load("./vectorizer/tripadvisor/word2vec_300.model")
word2vec = Word2Vec.load('./prosa-w2v/prosa.vec')
tfidf = pickle.load(open('./vectorizer/tripadvisor/tfidf.pickle', 'rb'))
model_dbow = Doc2Vec.load("./vectorizer/tripadvisor/model_dbow.model")
model_dmc = Doc2Vec.load("./vectorizer/tripadvisor/model_dmc.model")
model_dmm = Doc2Vec.load("./vectorizer/tripadvisor/model_dmm.model")

def build_doc_Vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += np.append(model_dbow[word] * tfidf[word], model_dmm[word] * tfidf[word])
            count += 1
        except KeyError: 
            continue
    if count != 0:
        vec /= count
    return vec

def build_Vector(tokens, word_size, doc_size):
    doc_vec = build_doc_Vector(tokens, doc_size)
    vec = np.zeros((MAX_SEQUENCE_LENGTH - len(tokens), doc_size + word_size))
    for word in tokens:
        try:
            word_vec = np.append(doc_vec, word2vec[word])
            vec = np.append(vec, word_vec)
        except KeyError: 
            word_vec = np.append(doc_vec, np.zeros((1, word_size)))
            vec = np.append(vec, word_vec)
            continue
    vec.reshape(MAX_SEQUENCE_LENGTH, doc_size + word_size)
    return vec

In [11]:
train_vecs = np.concatenate([[build_Vector(z, 500, 200)] for z in tqdm(map(lambda x: x.words, x_train))])
val_vecs = np.concatenate([[build_Vector(z, 500, 200)] for z in tqdm(map(lambda x: x.words, x_validation))])

0it [00:00, ?it/s]C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
935it [00:02, 328.51it/s]
100it [00:00, 402.33it/s]


In [12]:
batch_size = 256
num_epochs = 10
hidden_size = 10
timesteps = MAX_SEQUENCE_LENGTH
num_class = 1

In [13]:
num_data = len(train_vecs)
num_data_val = len(val_vecs)

train_vecs = train_vecs.reshape((num_data, timesteps, data_dim))
y_train = y_train.reshape((num_data, num_class))
val_vecs = val_vecs.reshape((num_data_val, timesteps, data_dim))
y_validation = y_validation.reshape((num_data_val, num_class))

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  import sys


In [14]:
model = Sequential()
model.add(Bidirectional(GRU(hidden_size, input_shape=(timesteps, data_dim)), merge_mode='concat'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_vecs, to_categorical(y_train), epochs=num_epochs, validation_data=(val_vecs, to_categorical(y_validation)))

Train on 935 samples, validate on 100 samples
Epoch 1/10
935/935 [==============================] - ETA: 1:57 - loss: 1.4761 - acc: 0.250 - ETA: 1:00 - loss: 1.2846 - acc: 0.375 - ETA: 40s - loss: 1.2496 - acc: 0.333 - ETA: 30s - loss: 1.2549 - acc: 0.32 - ETA: 24s - loss: 1.2239 - acc: 0.32 - ETA: 20s - loss: 1.1931 - acc: 0.35 - ETA: 17s - loss: 1.1959 - acc: 0.34 - ETA: 15s - loss: 1.1888 - acc: 0.34 - ETA: 13s - loss: 1.1770 - acc: 0.34 - ETA: 11s - loss: 1.1755 - acc: 0.33 - ETA: 10s - loss: 1.1706 - acc: 0.34 - ETA: 9s - loss: 1.1628 - acc: 0.3542 - ETA: 8s - loss: 1.1590 - acc: 0.367 - ETA: 7s - loss: 1.1493 - acc: 0.377 - ETA: 6s - loss: 1.1466 - acc: 0.381 - ETA: 6s - loss: 1.1403 - acc: 0.384 - ETA: 5s - loss: 1.1333 - acc: 0.393 - ETA: 4s - loss: 1.1282 - acc: 0.399 - ETA: 4s - loss: 1.1200 - acc: 0.407 - ETA: 3s - loss: 1.1253 - acc: 0.406 - ETA: 3s - loss: 1.1219 - acc: 0.407 - ETA: 2s - loss: 1.1216 - acc: 0.407 - ETA: 2s - loss: 1.1227 - acc: 0.406 - ETA: 1s - loss: 1.11

935/935 [==============================] - ETA: 5s - loss: 0.5958 - acc: 0.750 - ETA: 5s - loss: 0.5869 - acc: 0.765 - ETA: 5s - loss: 0.5303 - acc: 0.802 - ETA: 5s - loss: 0.5144 - acc: 0.820 - ETA: 4s - loss: 0.4956 - acc: 0.818 - ETA: 4s - loss: 0.4802 - acc: 0.833 - ETA: 4s - loss: 0.4598 - acc: 0.852 - ETA: 4s - loss: 0.4585 - acc: 0.843 - ETA: 4s - loss: 0.4604 - acc: 0.843 - ETA: 3s - loss: 0.4531 - acc: 0.850 - ETA: 3s - loss: 0.4597 - acc: 0.849 - ETA: 3s - loss: 0.4683 - acc: 0.841 - ETA: 3s - loss: 0.4747 - acc: 0.841 - ETA: 3s - loss: 0.4864 - acc: 0.830 - ETA: 2s - loss: 0.4815 - acc: 0.833 - ETA: 2s - loss: 0.4832 - acc: 0.835 - ETA: 2s - loss: 0.4805 - acc: 0.840 - ETA: 2s - loss: 0.4809 - acc: 0.843 - ETA: 2s - loss: 0.4807 - acc: 0.842 - ETA: 1s - loss: 0.4877 - acc: 0.837 - ETA: 1s - loss: 0.4985 - acc: 0.831 - ETA: 1s - loss: 0.5013 - acc: 0.829 - ETA: 1s - loss: 0.5003 - acc: 0.834 - ETA: 1s - loss: 0.5013 - acc: 0.834 - ETA: 0s - loss: 0.4991 - acc: 0.835 - ETA: 0s

In [15]:
# model.save('./model/bi_gru_3_pv/bi_gru_model_01.h5')  

In [16]:
# model = load_model('./model/bi_gru_3_pv/bi_gru_model_01.h5')
y_pred = model.predict(val_vecs)
y_pred = np.argmax(y_pred, axis=1)
print(classification_report(y_validation, y_pred, labels = [0, 1, 2], digits=8))

             precision    recall  f1-score   support

          0  0.58181818 0.80000000 0.67368421        40
          1  0.50000000 0.30000000 0.37500000        20
          2  0.69696970 0.57500000 0.63013699        40

avg / total  0.61151515 0.61000000 0.59652848       100

